# Nội dung chính 

1. Giới thiệu
2. Cài đặt
3. Khái niệm chính <br/>
    3.1: Selectors <br/>
    3.2: Request và Response <br/>
    3.3: Command line <br/>
    3.4: Spiders <br/>
    

# 1. Scrapy là gì

Scrapy là một framework được viết bằng Python, nó cấp sẵn 1 cấu trúc tương đối hoàn chỉnh để thực hiện việc crawl và extract data từ website một cách nhanh chóng và dễ dàng. Ví dụ như lấy toàn bộ hình ảnh trên 1 website; các bài viết trên các trang báo; thông tin dữ liệu các sản phẩm moblie, ô tô; các thông tin public trên facebook;...

Người dùng chỉ cần bổ sung thêm định nghĩa về dữ liệu cần lấy là xong, ví dụ như URL bắt đầu là gì, link chuyển qua trang mới, các thông tin cần lấy ở mỗi trang là gì...

# 2. Cài đặt

Scrapy chạy trên Python 2.7 và Python 3.5 trở lên trong CPython và PyPy 5.9 trở lên.

Nếu như bạn sử dụng Anaconda hoặc Miniconda, bạn có thể cài đặt gói từ conda-forge channel. Để cài đặt Scrapy sử dụng `conda`, chạy:

`conda install -c conda-forge scrapy`

Ngoài ra, nếu bạn đã quen với việc cài đặt các gói Python, bạn có thể cài đặt Scrapy từ PyPI với:

`pip install Scrapy`

# 3. Các khái niệm cơ bản

<b>Kiến trúc Scrapy:<b/><br/>
<img src="kientruc.png">

Để tạo project scrapy thì ta sử dụng lệnh: `scrapy startproject <nameproject>`.<br/>
Đây là cấu trúc thư mục khi vừa tạo thông qua scrapy:

`scraper/
├── scraper                         # nơi chứa code của dự án
│   ├── __init__.py
│   ├── items.py                    # nơi định nghĩa các trường dữ liệu cần lưu vào db
│   ├── pipelines.py                # nơi xử lý các item trích xuất được và lưu vào db
│   ├── settings.py                 # cấu hình thêm các phần mở rộng (middlewares) và các thông số cấu hình khác
│   └── spiders                     # thư mục chứa các spider
│       └── __init__.py
└── scrapy.cfg                      # file cấu hình về deploy và settings của project`

## 3.1: Selector

Khi quét các trang web, tác vụ phổ biến nhất cần thực hiện là trích xuất dữ liệu từ nguồn HTML. Scrapy có cơ chế riêng để trích xuất dữ liệu. Chúng chọn một số thành phần nhất định của tài liệu HTML được chỉ định bởi các biểu thức XPath hoặc CSS. XPath để chọn các nút trong tài liệu XML, cũng có thể được sử dụng cho các tài liệu HTML. CSS là ngôn ngữ để áp dụng các kiểu cho các tài liệu HTML.

Truy vấn các phản hồi bằng XPath và CSS ta sử dụng:  `response.xpath()` và `response.css()`

Ví dụ:
<br/>
`>>> response.xpath('//span/text()').get()`
<br/>
'good'
<br/>
`>>> response.css('span::text').get()`
<br/>
'good'


### Một số phương thức

Khi trích xuất dữ liệu ta sử dụng một số phương thức để trả về dữ liệu theo ý muốn. Có thể là trả về một mảng dữ liệu hay là phần tử đầu tiên của mảng dữ liệu.

<img src="method_extract.PNG">

### CSS Selectors

Xét ví dụ với một đoạn HTML dưới đây.

In [1]:
from scrapy import Selector
sel = Selector(text="""
<html>
 <head>
  <base href='http://example.com/' />
  <title>Example website</title>
 </head>
 <body>
  <div id='images'>
   <a href='image1.html'>Name: My image 1 <br /><img src='image1_thumb.jpg' /></a>
   <a href='image2.html'>Name: My image 2 <br /><img src='image2_thumb.jpg' /></a>
   <a href='image3.html'>Name: My image 3 <br /><img src='image3_thumb.jpg' /></a>
   <a href='image4.html'>Name: My image 4 <br /><img src='image4_thumb.jpg' /></a>
   <a href='image5.html'>Name: My image 5 <br /><img src='image5_thumb.jpg' /></a>
  </div>
 </body>
</html>""")

<i><b>Chú ý:</b> Khi thực hiện crawl trên web thì phần text của response chính là mã nguồn đc scrapy tải xuống  hay nói cách khác response = sel. </i>

 <b>title::text</b>: Trích chọn text trong thẻ title

In [2]:
sel.css('title::text').extract_first()

'Example website'

<b>*::text</b> chọn tất cả text ở trong các thẻ

In [3]:
sel.css('#images *::text').getall()

['\n   ',
 'Name: My image 1 ',
 '\n   ',
 'Name: My image 2 ',
 '\n   ',
 'Name: My image 3 ',
 '\n   ',
 'Name: My image 4 ',
 '\n   ',
 'Name: My image 5 ',
 '\n  ']

<b>foo::text</b> trả về kết quả không nếu phần tử foo tồn tại nhưng không chứa text.
<br/>
Sử dụng <b>default=''</b> nếu bạn muốn trả về string.

In [4]:
sel.css('img::text').getall()

[]

In [5]:
sel.css('img::text').get(default='')

''

<b>a::attr(href)</b> chọn giá trị thuộc tính href của links:

In [6]:
sel.css('a::attr(href)').getall()

['image1.html', 'image2.html', 'image3.html', 'image4.html', 'image5.html']

Trích xuất thông tin trong thẻ div có id = images:

In [9]:
sel.css('div#images').extract()

['<div id="images">\n   <a href="image1.html">Name: My image 1 <br><img src="image1_thumb.jpg"></a>\n   <a href="image2.html">Name: My image 2 <br><img src="image2_thumb.jpg"></a>\n   <a href="image3.html">Name: My image 3 <br><img src="image3_thumb.jpg"></a>\n   <a href="image4.html">Name: My image 4 <br><img src="image4_thumb.jpg"></a>\n   <a href="image5.html">Name: My image 5 <br><img src="image5_thumb.jpg"></a>\n  </div>']

### XPath Selectors

Chúng ta có thể chọn lọc ra một số thành phần chính xác trên một trang web bằng cách sử dụng XPath. Có thể lấy XPath dễ dàng bằng tool hay sử dụng Chrome để tìm XPath.

Ví dụ chúng ta sẽ lấy XPath trên stack overflow. Click chuột phải vào câu hỏi đầu tiên và chọn kiểm tra.

<img src="xp_0.PNG">

Bây giờ, chúng ta lấy XPath của phần tử đầu tiên `<div class="summary">`, kết quả sẽ tương tự `//*[@id="question-summary-59818222"]/div[2]`. 

<img src="xp_1.PNG">

Chúng ta sẽ test trong Javascript console.

<img src="xp_2.PNG">

Qua ví dụ trên ta có thể dễ dàng lấy XPath của bất kỳ phần tử nào

## 3.2: Request và Response

Scrapy sử dụng đối tượng Request và Response để thu thập dữ liệu websites.<br/>
Thông thường các đối tượng Request được tạo trong spider và truyền qua hệ thống cho đến khi đến trình download, thực thi yêu cầu và trả về đối tượng Response di chuyển trở lại spider đã gửi Request.

## 3.2: Command line tool

Scrapy được điều khiển thông qua command-line tool. Scrapy cung cấp một số lệnh, với nhiều mục đích thì sẽ sử dụng các nhóm lệnh khác nhau.

Một số lệnh command-line:

<img src="commandline.PNG">

Ví dụ về sử dụng scrapy shell. Đầu tiên mở cmd chạy command `scrapy shell [url]`

<img src="shell.PNG">
<br/>
<img src="shell1.PNG">

Sau đó có thể trực tiếp sử dụng các Selector để trích xuất trực tiếp, thực hiện điều này giúp kiểm tra một cách trực quan hơn. Ví dụ ta muốn tách title của bài báo, nhập `response.css('title::text').get()`

<img src="shell2.PNG">

## 3.4: Spiders

Spiders là class được viết bởi người dùng, xác định một hoặc một nhóm trang web nhất định sẽ được quét, bao gồm cách thực hiện thu thập thông tin và cách trích xuất dữ liệu có cấu trúc từ các trang web đó.

Đối với Spiders, quá trình thu thập dữ liệu có chu trình như sau:

1. Bạn bắt đầu bằng cách tạo các yêu cầu bạn đầu để thu thập dữ liệu từ URL đầu tiên và chỉ định chức năng gọi lại được gọi với phản hồi từ các yêu cầu đó.

2. Trong hàm gọi lại, bạn phân tích cú pháp phản hồi và trả về các ký tự với dữ liệu được trích xuất.  Những yêu cầu đó cũng có thể chứa một yêu cầu gọi lại, sau đó cũng sẽ được Scrapy duyệt và phản hồi lại yêu cầu đó.

3. Cuối cùng, các mục được trả về từ spiders sẽ được duy trì trong CSDL hoặc đươc ghi vào một tệp bằng cách sử dụng Feed exports

### items.py

Chứa các định nghĩa thông tin mình muốn extract. Giả sử tạo thư mục vnexpress, ta định nghĩa item VnexpressItem trong file items.py


In [10]:
from scrapy.item import Item, Field

class VnexpressItem(Item):
    # define the fields for your item here like:
    title = Field()
    url = Field()
    content = Field()
    author = Field()
    description = Field()
    publish_date = Field()
    

### scrapy.Spider

Định nghĩa cách chuyển giữa các trang và cách lấy thông tin. ví dụ định nghĩa domain thực hiện việc scrape.

Một số phương thức và thuộc tính chính:

<img src="method_scrapy.PNG">

Ví dụ: Crawl dữ liệu trên báo vnexpress 
<br/>
<a href="https://vnexpress.net/du-lich/viet-nam-huong-den-bau-troi-mo-asean-4001184.html">link báo</a>

Trước tiên ta tạo một file myspider.py trong thư mục spiders đã được tạo. Thư mục này là nơi chúng ta đưa ra các chỉ định cho Scrapy biết chính xác chúng ta muốn thu thập dữ liệu gì. Trong thư mục này các bạn có thể định nghĩa các spiders khác nhau cho các trang web khác nhau. 

Nếu muốn trả về item thì ta phải import item mà ta đã định nghĩa trong file items.py trong project. Dữ liệu xuất ra được sắp xếp theo key của mỗi item, sắp xếp theo thứ tự abc.

In [12]:
import scrapy
#from vnexpress.items import VnexpressItem

class MySpider(scrapy.Spider):
    name = "vnexpress"

    def start_requests(self):
        urls = ['https://vnexpress.net/du-lich/viet-nam-huong-den-bau-troi-mo-asean-4001184.html']
        for url in urls:
            #Phương thức call back gọi lại hàm parse_article khi gửi request đến url
            #hàm parse_article sẽ xử lý phản hồi đã được tải khi request url 
            request = scrapy.Request(url=url, callback=self.parse_artilce) 
            request.meta['url'] = url
            yield request

    def parse_artilce(self, response):
        title = response.xpath('//*[@id="col_sticky"]/h1/text()').extract()[0].strip()
        description = response.xpath('//*[@id="col_sticky"]/p[1]').extract()[0].strip()
        content = response.xpath('//*[@id="col_sticky"]/article').extract()[0].strip()
        author = response.xpath('//*[@id="col_sticky"]/p[2]/strong/text()').extract()[0].strip()
        publish_date = response.xpath('//*[@id="col_sticky"]/header/span/text()').extract()[0].strip()
        
        item = VnexpressItem()
        item['title'] = title
        item['url'] = response.meta['url']
        item['content'] = content
        item['description'] = description
        item['author'] = author
        item['publish_date'] = publish_date
        
        yield item
        

Sau đó sẽ chạy lệnh ở command line để thực hiện lấy các thông tin mà spider trích xuất.

`scrapy crawl vnexpress`

Nếu muốn xuất ra file json thì sử dụng:<br/>
<br/>
`scrapy crawl vnexpress -o output.json`

Ở đây sử dụng request.meta và response.meta, trong đó response.meta trả về giá trị lưu ở request.meta trước khi request url. Hay nói cách khác đó là: truyền đối số cho các hàm có thể gọi được và nhận các đối số đó trong cuộc gọi lại thứ hai.


### Pipelines.py

Để Insert các kết quả tìm được vào database. 

# Bài tập

trích xuất tên, loại, giá, nơi sản xuất, cỡ của tất cả các loại giày ở trang web https://adidasstore.vn/ rồi lưu lại kết quả vào file adidas.json.<br/>
Chú ý: loại là giày adidasneo hay adidas nam hay adidas nữ.

Tham khảo thêm tại https://github.com/kieulydt/MobileCrawler :v :v :v

Bài báo cáo còn nhiều sai sót và hổng do bản tính lười biếng, mong thí chủ lượng thứ :((((